# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770880510890                   -0.52    9.0         
  2   -2.772148760382       -2.90       -1.32    1.0    160ms
  3   -2.772170270126       -4.67       -2.43    1.0    139ms
  4   -2.772170652720       -6.42       -3.15    1.0    156ms
  5   -2.772170722888       -7.15       -4.36    2.0    160ms
  6   -2.772170722970      -10.09       -4.63    1.0    157ms
  7   -2.772170723013      -10.37       -5.72    1.0    146ms
  8   -2.772170723015      -11.69       -6.25    2.0    180ms
  9   -2.772170723015      -13.57       -6.72    1.0    152ms
 10   -2.772170723015   +  -14.57       -7.25    1.0    180ms
 11   -2.772170723015   +  -14.51       -7.93    2.0    184ms
 12   -2.772170723015      -14.75       -8.32    1.0    184ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735578504253355

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770839547434                   -0.52    9.0         
  2   -2.772062866329       -2.91       -1.32    1.0    137ms
  3   -2.772083091869       -4.69       -2.46    1.0    178ms
  4   -2.772083350579       -6.59       -3.17    1.0    141ms
  5   -2.772083417327       -7.18       -4.12    2.0    182ms
  6   -2.772083417634       -9.51       -4.41    1.0    162ms
  7   -2.772083417808       -9.76       -5.39    1.0    149ms
  8   -2.772083417810      -11.64       -5.83    1.0    159ms
  9   -2.772083417811      -12.88       -6.32    1.0    161ms
 10   -2.772083417811      -13.56       -7.88    2.0    183ms
 11   -2.772083417811      -14.40       -7.80    2.0    174ms
 12   -2.772083417811      -14.45       -8.54    1.0    161ms

Polarizability via ForwardDiff:       1.7725349768435497
Polarizability via finite difference: 1.7735578504253355
